# Day 1 - API Access

This is just an introductory notebook to make sure you have API access. We simply test each API and ensure we're getting a response. If you run into any issues here, please reach out to us!

Please make sure that you've set your environment variables in the `.env` file at the root of this repositiory. Specifically, this notebook (and the others in this folder) require the following variables to be set: `OPENAI_API_KEY`, `GOOGLE_APPLICATION_CREDENTIALS`

In [1]:
# Load environment variables
import dotenv
dotenv.load_dotenv("../../.env", override=True)

True

### Chat Completion APIs vs Text Completion APIs

Both OpenAI and Google have two different APIs we can use to make calls to their respective LLMs. Chat Completion APIs are conversational in nature, meaning the user and the model take turns to send messages (like chatGPT). Text Completion APIs on the other hand are meant to be singular prompts where we give a prompt and the model attempts to "complete" the text sequence.

Chat Completion APIs: [ChatCompletion (OpenAI)](https://platform.openai.com/docs/api-reference/chat), [ChatModel (Google)](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#chat_model_parameters)

Text Completion APIs: [Completion (OpenAI)](https://platform.openai.com/docs/api-reference/completions), [TextGeneration (Google)](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#text_model_parameters)

Note that the latest OpenAI models such as GPT-4 and GPT-3.5-turbo are only available through their chat completion API. Thus we exclusively use only the chat completion API for OpenAI models. For Google's PaLM models, we primarily use only the text completion API. We use the chat completion API only for the chatbot example. This is based on [Google's](https://developers.generativeai.google/models/language) descriptions of the uses for each model.

## OpenAI

In [7]:
import openai

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "How do I make a margherita pizza?"},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages 
)

print(messages[-1]["content"])
print(chatbot_response.choices[0].message["content"])

How do I make a margherita pizza?
To make a Margherita pizza, follow these steps:

Ingredients:

1) Pizza Dough:
- 2 1/4 teaspoons (or 1 packet) active dry yeast
- 1 1/3 cups warm water (around 110°F/45°C)
- 3 1/2 cups all-purpose flour
- 2 tablespoons olive oil
- 2 teaspoons sugar
- 1 teaspoon salt

2) Toppings:
- 1 cup tomato sauce (preferably San Marzano or homemade)
- 8 oz. fresh mozzarella cheese (sliced or torn into small pieces)
- 1/2 cup fresh basil leaves
- 2 tablespoons olive oil
- Salt, to taste

Instructions:

1. Prepare the pizza dough:
  a. In a small bowl, combine the warm water and sugar. Sprinkle the active dry yeast on top and let it sit for about 5 minutes, until it becomes foamy.
  b. In a large mixing bowl, combine the flour and salt. Make a well in the middle and pour in the yeast mixture and olive oil. Mix the ingredients together until a rough dough forms.
  c. Knead the dough on a floured surface for about 10 minutes, or until it becomes smooth and elastic. Alt

## Google PaLM

In [8]:
from vertexai.preview.language_models import TextGenerationModel

QUESTION = "How do I make a margherita pizza?"

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(
    QUESTION,
    temperature=0.3,
    max_output_tokens=256,
)

print(QUESTION)
print("---" * 30)
print(response.text)

How do I make a margherita pizza?
------------------------------------------------------------------------------------------
Ingredients:

* 1 batch of pizza dough (homemade or store-bought)
* 1 cup pizza sauce
* 2 cups shredded mozzarella cheese
* 2-3 large fresh tomatoes, sliced
* 1/2 cup fresh basil leaves, torn
* 1/4 cup olive oil
* Salt and pepper to taste

Instructions:

1. Preheat the oven to 450 degrees F (230 degrees C).
2. If using homemade pizza dough, prepare the dough according to the recipe. If using store-bought pizza dough, let it come to room temperature for about 30 minutes before using.
3. Roll out the pizza dough into a 12-inch circle.
4. Spread the pizza sauce evenly over the dough.
5. Top with the mozzarella cheese, sliced tomatoes, and basil leaves.
6. Drizzle with olive oil and season with salt and pepper to taste.
7. Bake the pizza for 15-20 minutes, or until the crust is golden brown and the cheese is melted and bubbly.
8. Let the pizza cool for a few minutes 

## Self-Hosted (TBD)